In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ==============================================================================
# HAAST: A Novel Multimodal Transformer Architecture
# ==============================================================================
# Complete implementation in a single file for Kaggle environments.
#
# This script includes:
# 1.  All necessary library imports, now with tqdm.
# 2.  Centralized configuration management.
# 3.  Modality-specific encoders (Text, Vision) with gating.
# 4.  Core HAA Block components, including Associative Attention and MoE.
# 5.  The main HAAST model assembling all modules.
# 6.  Output heads for different tasks.
# 7.  A training and evaluation class with progress bars, mixed-precision, and pruning.
# 8.  A main execution block to run a full training and testing pipeline on CIFAR-10.
# ==============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.nn.utils import prune
from torch.utils.data import DataLoader
from torch.utils.checkpoint import checkpoint
import math
import yaml
import unittest
from einops.layers.torch import Rearrange
import torchvision
import torchvision.transforms as transforms
import time
from tqdm import tqdm # Import tqdm

# ==============================================================================
# 1. CONFIGURATION (Originally haast/config.py)
# ==============================================================================
def get_config():
    """Returns the default configuration for the HAAST model."""
    # Aligned dimensions to multiples of 32 for TensorCore efficiency
    return {
        # Model Architecture
        "d_model": 768,
        "n_blocks": 12,
        "n_heads": 12,
        "vocab_size": 32000,
        "experts_per_block": 4,
        "top_k_experts": 2,
        "global_sync_interval": 4,

        # Vision Encoder
        "image_size": 224,
        "patch_size": 16,

        # Training & Efficiency
        "dropout": 0.1,
        "optimizer": "AdamW",
        "lr_schedule": {
            "warmup_pct": 0.05,
            "decay": "cosine"
        },
        "pruning_amount": 0.0, # Set to > 0 to enable pruning during training
        "mixed_precision": True,
        "gradient_checkpointing": True,

        # Task Specific
        "num_classes": 10, # For CIFAR-10 classification

        # Not used in this version but part of the original architecture
        "kernel_features": 256,
        "memory_capacity": 1024,
        "early_exit_threshold": 0.9,
        "sparse_attention_k": 32,
    }

# ==============================================================================
# 2. SHARED UTILITIES & LAYERS
# ==============================================================================
class GatingLayer(nn.Module):
    """A sparse gating layer to prune activations."""
    def __init__(self, d_model: int, prune_ratio: float = 0.3):
        super().__init__()
        self.gate = nn.Linear(d_model, d_model)
        self.prune_ratio = prune_ratio

    def forward(self, x):
        gate_scores = torch.sigmoid(self.gate(x))
        if self.training and self.prune_ratio > 0:
            # Keep top (1 - prune_ratio) % of activations
            k = int(gate_scores.numel() * (1 - self.prune_ratio))
            # Find the threshold for the top-k values
            threshold, _ = torch.kthvalue(gate_scores.view(-1), k)
            mask = gate_scores >= threshold
        else:
            # During inference, use a fixed threshold or no pruning
            mask = gate_scores > self.prune_ratio
        return x * mask

# ==============================================================================
# 3. ENCODERS (Originally haast/encoders/)
# ==============================================================================
# --- 3.1 Text Encoder ---
class HarmonicPositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [batch_size, seq_len, d_model]
        """
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class TextEncoder(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, dropout: float = 0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = HarmonicPositionalEncoding(d_model, dropout)
        self.gate = GatingLayer(d_model)
        self.d_model = d_model

    def forward(self, src: torch.Tensor) -> torch.Tensor:
        # src shape: [B, T]
        x = self.embedding(src) * math.sqrt(self.d_model)
        x = self.pos_encoder(x)
        x = self.gate(x)
        return x # Output: [B, T, d_model]

# --- 3.2 Vision Encoder ---
class VisionEncoder(nn.Module):
    def __init__(self, image_size: int, patch_size: int, d_model: int, channels: int = 3):
        super().__init__()
        image_height, image_width = image_size, image_size
        patch_height, patch_width = patch_size, patch_size
        assert image_height % patch_height == 0 and image_width % patch_width == 0
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_height * patch_width

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_size, p2=patch_size),
            nn.Linear(patch_dim, d_model),
        )
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, d_model))
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
        self.gate = GatingLayer(d_model)

    def forward(self, img: torch.Tensor) -> torch.Tensor:
        # img shape: [B, 3, H, W]
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape
        cls_tokens = self.cls_token.expand(b, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.gate(x)
        return x # Output: [B, num_patches + 1, d_model]

# ==============================================================================
# 4. UNIFIED LATENT PROJECTION (Originally haast/encoder_utils.py)
# ==============================================================================
class UnifiedLatentProjection(nn.Module):
    def __init__(self, d_model: int, num_modalities: int = 2):
        super().__init__()
        self.projection = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.fusion_weights = nn.Parameter(torch.ones(num_modalities))

    def forward(self, *encoder_outputs: torch.Tensor, mode: str = 'concat') -> torch.Tensor:
        if mode == 'concat':
            x = torch.cat(encoder_outputs, dim=1)
        elif mode in ('mean_pool_sum', 'weighted_pool_sum'):
            pooled_outputs = [torch.mean(output, dim=1) for output in encoder_outputs]
            if mode == 'weighted_pool_sum':
                normalized_weights = F.softmax(self.fusion_weights, dim=0)
                weighted_pooled = torch.stack([out * w for out, w in zip(pooled_outputs, normalized_weights)])
                fused_pooled = torch.sum(weighted_pooled, dim=0)
            else: # mean_pool_sum
                fused_pooled = torch.stack(pooled_outputs, dim=0).sum(dim=0)
            max_len = max(out.size(1) for out in encoder_outputs)
            x = fused_pooled.unsqueeze(1).repeat(1, max_len, 1)
        else:
            raise ValueError(f"Unknown or unsupported fusion mode: {mode}")

        x = self.projection(x)
        x = self.norm(x)
        return x

# ==============================================================================
# 5. CORE HAA BLOCK (Originally haast/blocks/)
# ==============================================================================
# --- 5.1 Associative Attention ---
class AssociativeAttention(nn.Module):
    def __init__(self, d_model: int, n_heads: int):
        super().__init__()
        assert d_model % n_heads == 0
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        
        self.to_qkv = nn.Linear(d_model, d_model * 3, bias=False)
        self.to_out = nn.Linear(d_model, d_model)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b, n, _, h = *x.shape, self.n_heads
        q, k, v = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: Rearrange('b n (h d) -> b h n d', h=h)(t), (q, k, v))
        
        beta = self.head_dim ** -0.5
        dots = torch.einsum('bhid,bhjd->bhij', q, k) * beta
        attn = dots.softmax(dim=-1)
        
        out = torch.einsum('bhij,bhjd->bhid', attn, v)
        out = Rearrange('b h n d -> b n (h d)')(out)
        return self.to_out(out)

# --- 5.2 Mixture-of-Experts Feed-Forward ---
class MoEFeedForward(nn.Module):
    def __init__(self, d_model: int, n_experts: int, top_k: int, dropout: float):
        super().__init__()
        self.router = nn.Linear(d_model, n_experts)
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(d_model, 4 * d_model),
                nn.GELU(),
                nn.Dropout(dropout),
                nn.Linear(4 * d_model, d_model)
            ) for _ in range(n_experts)
        ])
        self.top_k = top_k

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b, n, d = x.shape
        x_flat = x.view(-1, d)
        
        router_logits = self.router(x_flat)
        routing_weights, selected_experts = torch.topk(
            F.softmax(router_logits, dim=-1), self.top_k, dim=-1
        )
        routing_weights /= routing_weights.sum(dim=-1, keepdim=True)
        
        final_output = torch.zeros_like(x_flat)
        expert_mask = F.one_hot(selected_experts, num_classes=len(self.experts)).permute(2, 0, 1)

        for i, expert in enumerate(self.experts):
            idx, top_x = torch.where(expert_mask[i])
            if top_x.shape[0] == 0:
                continue
            weights = routing_weights[idx, top_x].unsqueeze(-1)
            final_output.index_add_(0, idx, weights * expert(x_flat[idx]))
            
        return final_output.view(b, n, d)

# --- 5.3 Full HAA Block ---
class HAABlock(nn.Module):
    def __init__(self, config: dict):
        super().__init__()
        d_model = config['d_model']
        n_heads = config['n_heads']
        dropout = config['dropout']
        
        self.use_checkpoint = config.get("gradient_checkpointing", True)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.assoc_attn = AssociativeAttention(d_model, n_heads)
        
        self.norm_moe = nn.LayerNorm(d_model)
        self.moe_ffn = MoEFeedForward(d_model, config['experts_per_block'], config['top_k_experts'], dropout)
        self.dropout = nn.Dropout(dropout)

    def _checkpointable_forward(self, module, norm, data):
        return module(norm(data))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Pre-norm architecture
        attn_residual = x
        if self.use_checkpoint and self.training:
            attn_out = checkpoint(self._checkpointable_forward, self.assoc_attn, self.norm1, x)
        else:
            attn_out = self.assoc_attn(self.norm1(x))
        x = attn_residual + self.dropout(attn_out)
        
        moe_residual = x
        if self.use_checkpoint and self.training:
            moe_out = checkpoint(self._checkpointable_forward, self.moe_ffn, self.norm_moe, x)
        else:
            moe_out = self.moe_ffn(self.norm_moe(x))
        x = moe_residual + self.dropout(moe_out)
        
        return x

# ==============================================================================
# 6. GLOBAL SYNC LAYER (Originally haast/sync.py)
# ==============================================================================
class GlobalSync(nn.Module):
    def __init__(self, d_model: int):
        super().__init__()
        self.gate = nn.Sequential(nn.Linear(d_model, 1), nn.Sigmoid())
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        global_summary = torch.mean(x, dim=1, keepdim=True)
        gate_val = self.gate(global_summary)
        return x * gate_val

# ==============================================================================
# 7. OUTPUT HEADS (Originally haast/heads/)
# ==============================================================================
class ClassificationHead(nn.Module):
    def __init__(self, d_model: int, num_classes: int):
        super().__init__()
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        cls_token_representation = x[:, 0]
        return self.classifier(self.norm(cls_token_representation))

class LanguageModelHead(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.norm = nn.LayerNorm(d_model)
        self.decoder = nn.Linear(d_model, vocab_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.decoder(self.norm(x))

# ==============================================================================
# 8. HAAST MAIN MODEL (Originally haast/model.py)
# ==============================================================================
class HAAST(nn.Module):
    def __init__(self, config: dict):
        super().__init__()
        self.config = config
        
        # --- Encoders ---
        self.text_encoder = TextEncoder(config['vocab_size'], config['d_model'], config['dropout'])
        self.vision_encoder = VisionEncoder(config['image_size'], config['patch_size'], config['d_model'])
        
        # --- Fusion Layer ---
        self.latent_projection = UnifiedLatentProjection(config['d_model'])
        
        # --- Core Blocks ---
        self.blocks = nn.ModuleList([
            HAABlock(config) for _ in range(config['n_blocks'])
        ])
        
        # --- Sync Layer ---
        self.global_sync = GlobalSync(config['d_model'])
        self.sync_interval = config['global_sync_interval']
        
        # --- Output Head ---
        self.output_head = ClassificationHead(config['d_model'], num_classes=config['num_classes'])

    def forward(self, text_input=None, vision_input=None) -> torch.Tensor:
        encoder_outputs = []
        if text_input is not None:
            encoder_outputs.append(self.text_encoder(text_input))
        if vision_input is not None:
            encoder_outputs.append(self.vision_encoder(vision_input))
        
        if not encoder_outputs:
            raise ValueError("At least one input modality must be provided.")
            
        x = self.latent_projection(*encoder_outputs, mode='concat')
        
        for i, block in enumerate(self.blocks):
            x = block(x)
            if (i + 1) % self.sync_interval == 0:
                x = self.global_sync(x)
                
        return self.output_head(x)

# ==============================================================================
# 9. DATA LOADING FOR REAL DATASET (CIFAR-10)
# ==============================================================================
def get_dataloaders(image_size: int, batch_size: int):
    """Downloads and prepares CIFAR-10 dataloaders."""
    print("Preparing CIFAR-10 dataloaders...")
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize for CIFAR-10
    ])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=2, pin_memory=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             shuffle=False, num_workers=2, pin_memory=True)
    print("✅ Dataloaders ready.")
    return trainloader, testloader

# ==============================================================================
# 10. TRAINING & EFFICIENCY (Originally haast/training/)
# ==============================================================================
class Trainer:
    def __init__(self, model: nn.Module, optimizer, scheduler, device, config: dict):
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.config = config
        self.scaler = GradScaler(enabled=config.get('mixed_precision', True))
        
    def train_epoch(self, dataloader, epoch_num, total_epochs):
        self.model.train()
        total_loss = 0
        
        # Wrap dataloader with tqdm for a progress bar
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch_num}/{total_epochs} [Training]")
        
        # FIX: Use enumerate to get the batch index `i`
        for i, batch in enumerate(progress_bar):
            images, targets = batch
            images = images.to(self.device, non_blocking=True)
            targets = targets.to(self.device, non_blocking=True)

            self.optimizer.zero_grad(set_to_none=True)

            with autocast(enabled=self.scaler.is_enabled()):
                # We are training on vision-only, so pass only vision_input
                outputs = self.model(vision_input=images)
                loss = F.cross_entropy(outputs, targets)

            self.scaler.scale(loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
            
            total_loss += loss.item()
            
            # FIX: Calculate average loss using the batch index (i + 1)
            progress_bar.set_postfix(loss=total_loss/(i + 1))

        # Pruning is done once per epoch after the weights have been updated
        self.prune_model()
        return total_loss / len(dataloader)

    def evaluate(self, dataloader):
        self.model.eval()
        total_loss = 0
        correct = 0
        total = 0
        
        # Wrap dataloader with tqdm for a progress bar
        progress_bar = tqdm(dataloader, desc="Evaluating")

        with torch.no_grad():
            for batch in progress_bar:
                images, targets = batch
                images = images.to(self.device, non_blocking=True)
                targets = targets.to(self.device, non_blocking=True)

                with autocast(enabled=self.scaler.is_enabled()):
                    outputs = self.model(vision_input=images)
                    loss = F.cross_entropy(outputs, targets)

                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

                # Update progress bar with running accuracy
                progress_bar.set_postfix(accuracy=f"{100 * correct / total:.2f}%")

        avg_loss = total_loss / len(dataloader)
        accuracy = 100 * correct / total
        return avg_loss, accuracy


    def prune_model(self):
        pruning_amount = self.config.get('pruning_amount', 0.0)
        if not (pruning_amount > 0 and self.model.training):
            return

        print(f"Applying {pruning_amount:.1%} unstructured L1 pruning...")
        for module in self.model.modules():
            if isinstance(module, (nn.Linear, nn.Conv2d)):
                prune.l1_unstructured(module, name="weight", amount=pruning_amount)
                # To make pruning permanent and remove re-parameterization overhead
                prune.remove(module, 'weight')
        print("Pruning complete.")

# ==============================================================================
# 11. MAIN EXECUTION BLOCK FOR TRAINING & EVALUATION
# ==============================================================================
def main():
    """
    Main function to run the HAAST training and evaluation pipeline.
    """
    print("🚀 Starting HAAST Model Training & Evaluation on CIFAR-10...")

    # --- Training Configuration ---
    # For a full run, use more epochs. For a quick test, 3-5 is fine.
    EPOCHS = 10
    BATCH_SIZE = 32 # Adjust based on your GPU memory
    LEARNING_RATE = 1e-4

    # --- Get Model Config and Device ---
    config = get_config()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Reduce model size for quick CPU testing if needed
    if device.type == 'cpu':
        print("⚠️ CPU detected. Downscaling model and data for quick testing.")
        config['d_model'] = 128
        config['n_heads'] = 4
        config['n_blocks'] = 3
        config['experts_per_block'] = 2
        BATCH_SIZE = 16 # Reduce batch size for CPU memory

    # --- Data Loading ---
    train_loader, test_loader = get_dataloaders(config['image_size'], BATCH_SIZE)

    # --- Model Initialization ---
    try:
        model = HAAST(config).to(device)
        print("✅ Model initialization successful.")
        num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Model Parameters: {num_params / 1e6:.2f}M")
    except Exception as e:
        print(f"❌ Model initialization failed: {e}")
        return

    # --- Optimizer and Scheduler ---
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.05)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
    # --- Trainer Initialization ---
    trainer = Trainer(model, optimizer, scheduler, device, config)

    # --- Training & Evaluation Loop ---
    print("\n" + "="*50)
    print("🔥 Starting Training 🔥")
    print("="*50)

    for epoch in range(1, EPOCHS + 1):
        start_time = time.time()

        # Train one epoch
        train_loss = trainer.train_epoch(train_loader, epoch, EPOCHS)
        
        # Evaluate on the test set
        test_loss, test_accuracy = trainer.evaluate(test_loader)
        
        # Step the scheduler
        if trainer.scheduler:
            trainer.scheduler.step()

        epoch_duration = time.time() - start_time
        
        # The progress bar provides running feedback, so we print a summary at the end of the epoch
        print(f"Epoch {epoch}/{EPOCHS} Summary | Time: {epoch_duration:.2f}s")
        print(f"  -> Train Loss: {train_loss:.4f}")
        print(f"  -> Test Loss:  {test_loss:.4f}")
        print(f"  -> Test Accuracy: {test_accuracy:.2f}%")
        print(f"  -> Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        print("-" * 50)


    print("\n🎉 Training finished!")
    final_loss, final_accuracy = trainer.evaluate(test_loader)
    print(f"Final Test Accuracy: {final_accuracy:.2f}%")


if __name__ == '__main__':
    main()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from typing import List, Optional, Tuple
import math

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
import numpy as np

# ==========================================================================================
# |                                  Vision Transformer (ViT) Architecture                 |
# ==========================================================================================

class PatchEmbedding(nn.Module):
    """
    Splits an image into patches, flattens them, and projects them into a higher dimension.
    Also adds a learnable [CLS] token and positional embeddings.
    """
    def __init__(self, image_size: int, patch_size: int, in_channels: int, embed_dim: int):
        super().__init__()
        # Ensure image dimensions are divisible by patch size
        assert image_size % patch_size == 0, "Image dimensions must be divisible by the patch size."

        self.image_size = image_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.embed_dim = embed_dim

        # Calculate number of patches along one side
        self.num_patches_side = image_size // patch_size
        # Total number of patches
        self.num_patches = self.num_patches_side * self.num_patches_side

        # Convolutional layer to extract patches and project them
        # Kernel size and stride equal to patch_size effectively create non-overlapping patches
        self.projection = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

        # Learnable [CLS] token for classification
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        
        # Learnable positional embeddings for each patch + CLS token
        self.position_embeddings = nn.Parameter(torch.randn(1, self.num_patches + 1, embed_dim))

    def forward(self, x: Tensor) -> Tensor:
        B, C, H, W = x.shape

        # Validate input image size
        if H != self.image_size or W != self.image_size or C != self.in_channels:
            raise ValueError(
                f"Input image size ({B}x{C}x{H}x{W}) does not match model's configured size "
                f"(Bx{self.in_channels}x{self.image_size}x{self.image_size})."
            )

        # Apply convolution to get patch embeddings
        # Output shape will be (B, embed_dim, num_patches_side, num_patches_side)
        x = self.projection(x)
        
        # Flatten the spatial dimensions to get a sequence of patch embeddings
        # Output shape will be (B, embed_dim, num_patches)
        x = x.flatten(2) # Flatten HxW into a single dimension
        x = x.transpose(1, 2) # Transpose to (B, num_patches, embed_dim)

        # Expand CLS token for the current batch size
        cls_tokens = self.cls_token.expand(B, -1, -1) # (B, 1, embed_dim)

        # Concatenate CLS token with patch embeddings
        x = torch.cat((cls_tokens, x), dim=1) # (B, num_patches + 1, embed_dim)

        # Add positional embeddings
        x += self.position_embeddings

        return x

class MultiHeadSelfAttention(nn.Module):
    """
    Multi-Head Self-Attention mechanism.
    Allows the model to jointly attend to information from different representation
    subspaces at different positions.
    """
    def __init__(self, embed_dim: int, num_heads: int, dropout: float = 0.0):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads # Dimension of each attention head
        self.scale = self.head_dim ** -0.5 # Scaling factor for dot product attention

        # Linear layers for Query, Key, Value projections
        self.qkv_proj = nn.Linear(embed_dim, embed_dim * 3, bias=False)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: Tensor) -> Tensor:
        B, N, C = x.shape # Batch size, Sequence length, Embedding dimension

        # Project input to Q, K, V
        # Shape: (B, N, embed_dim * 3) -> split into (B, N, embed_dim) for Q, K, V
        qkv = self.qkv_proj(x).chunk(3, dim=-1)
        
        # Reshape Q, K, V for multi-head attention
        # (B, N, embed_dim) -> (B, N, num_heads, head_dim) -> (B, num_heads, N, head_dim)
        q, k, v = map(lambda t: t.reshape(B, N, self.num_heads, self.head_dim).transpose(1, 2), qkv)

        # Calculate attention scores (Q @ K_T)
        # (B, num_heads, N, head_dim) @ (B, num_heads, head_dim, N) -> (B, num_heads, N, N)
        attn_scores = (q @ k.transpose(-2, -1)) * self.scale
        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Apply attention weights to Values (attn_weights @ V)
        # (B, num_heads, N, N) @ (B, num_heads, N, head_dim) -> (B, num_heads, N, head_dim)
        attn_output = attn_weights @ v

        # Concatenate heads and project back to original embedding dimension
        # (B, num_heads, N, head_dim) -> (B, N, num_heads, head_dim) -> (B, N, embed_dim)
        attn_output = attn_output.transpose(1, 2).reshape(B, N, C)
        
        # Final linear projection
        output = self.out_proj(attn_output)
        output = self.dropout(output) # Apply dropout after output projection
        return output

class MLP(nn.Module):
    """
    A simple Multi-Layer Perceptron (Feed-Forward Network).
    Applied to each token independently.
    """
    def __init__(self, in_features: int, hidden_features: int, out_features: int, dropout: float = 0.0):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: Tensor) -> Tensor:
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

class TransformerEncoderBlock(nn.Module):
    """
    A single Vision Transformer encoder block.
    Consists of Multi-Head Self-Attention, Layer Normalization, and an MLP.
    Includes residual connections.
    """
    def __init__(self, embed_dim: int, num_heads: int, mlp_ratio: float = 4.0, dropout: float = 0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = MultiHeadSelfAttention(embed_dim, num_heads, dropout=dropout)
        self.norm2 = nn.LayerNorm(embed_dim)
        
        mlp_hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = MLP(embed_dim, mlp_hidden_dim, embed_dim, dropout=dropout)

    def forward(self, x: Tensor) -> Tensor:
        # LayerNorm -> Attention -> Add residual
        x = x + self.attn(self.norm1(x))
        
        # LayerNorm -> MLP -> Add residual
        x = x + self.mlp(self.norm2(x))
        return x

class VisionTransformer(nn.Module):
    """
    The complete Vision Transformer model for image classification.
    """
    def __init__(self, image_size: int, patch_size: int, in_channels: int, num_classes: int,
                 embed_dim: int, num_layers: int, num_heads: int, mlp_ratio: float = 4.0,
                 dropout: float = 0.1):
        super().__init__()
        self.patch_embedding = PatchEmbedding(image_size, patch_size, in_channels, embed_dim)
        
        # Stack of Transformer encoder blocks
        self.encoder = nn.Sequential(
            *[TransformerEncoderBlock(embed_dim, num_heads, mlp_ratio, dropout) 
              for _ in range(num_layers)]
        )
        
        self.final_norm = nn.LayerNorm(embed_dim) # Final LayerNorm before classification head
        self.classification_head = nn.Linear(embed_dim, num_classes) # Classification head

        self.apply(self._init_weights) # Apply custom weight initialization

    def _init_weights(self, module):
        """Custom weight initialization for better training stability."""
        if isinstance(module, nn.Linear):
            # Glorot / Xavier normal initialization for linear layers
            torch.nn.init.xavier_normal_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, nn.LayerNorm):
            nn.init.constant_(module.bias, 0)
            nn.init.constant_(module.weight, 1.0)
        elif isinstance(module, PatchEmbedding):
            # Initialize positional embeddings and CLS token
            nn.init.trunc_normal_(module.position_embeddings, std=0.02)
            nn.init.trunc_normal_(module.cls_token, std=0.02)
            # For patch projection (Conv2d), standard initialization is usually fine
            if isinstance(module.projection, nn.Conv2d):
                nn.init.kaiming_normal_(module.projection.weight, mode='fan_out', nonlinearity='relu')
                if module.projection.bias is not None:
                    nn.init.constant_(module.projection.bias, 0)


    def forward(self, x: Tensor) -> Tensor:
        # 1. Patch Embedding + Positional Encoding
        x = self.patch_embedding(x) # (B, num_patches + 1, embed_dim)

        # 2. Transformer Encoder Blocks
        x = self.encoder(x) # (B, num_patches + 1, embed_dim)

        # 3. Take the [CLS] token's output for classification
        cls_token_output = self.final_norm(x[:, 0]) # (B, embed_dim)

        # 4. Classification Head
        logits = self.classification_head(cls_token_output) # (B, num_classes)
        return logits

# ==========================================================================================
# |                                  Training and Evaluation                               |
# ==========================================================================================

if __name__ == "__main__":
    torch.autograd.set_detect_anomaly(True)

    print("🚀 Starting Vision Transformer Training on CIFAR-10...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # ViT Model Parameters (tuned for CIFAR-10, can be adjusted)
    vit_image_size = 32
    vit_patch_size = 4 # Smaller patches mean more tokens, potentially better for small images but more compute
    vit_in_channels = 3
    vit_num_classes = 10
    vit_embed_dim = 256 # Dimension of token embeddings
    vit_num_layers = 6  # Number of Transformer encoder blocks
    vit_num_heads = 8   # Number of attention heads
    vit_mlp_ratio = 4.0 # Expansion factor for the MLP
    vit_dropout = 0.1

    print("\n[STEP 1/5] Loading and preprocessing CIFAR-10 dataset...")
    # CIFAR-10 normalization values (mean and std for RGB channels)
    cifar10_mean = (0.4914, 0.4822, 0.4465)
    cifar10_std = (0.2471, 0.2435, 0.2616)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std)
    ])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True) # pin_memory for faster GPU transfer
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)
    print("✅ Dataset loaded.")

    print("\n[STEP 2/5] Initializing the Vision Transformer model...")
    model = VisionTransformer(
        image_size=vit_image_size,
        patch_size=vit_patch_size,
        in_channels=vit_in_channels,
        num_classes=vit_num_classes,
        embed_dim=vit_embed_dim,
        num_layers=vit_num_layers,
        num_heads=vit_num_heads,
        mlp_ratio=vit_mlp_ratio,
        dropout=vit_dropout
    ).to(device)
    print(f"✅ Model initialized with {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters.")

    print("\n[STEP 3/5] Setting up loss function and optimizer...")
    criterion = nn.CrossEntropyLoss()
    # AdamW is generally a good choice for Transformers
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01) # Slightly higher LR than MathNet
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50) # More epochs for CIFAR-10
    print("✅ Setup complete.")

    print("\n[STEP 4/5] Starting training loop...")
    num_epochs = 50 # Increased epochs for CIFAR-10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        progress_bar = tqdm(trainloader, desc=f"Epoch {epoch + 1}/{num_epochs} [Train]", leave=True)
        
        for i, data in enumerate(progress_bar):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            if torch.isnan(loss):
                print("🚨 Loss is NaN! Stopping training.")
                break
                
            loss.backward()
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) 
            optimizer.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}', 
                'lr': f'{scheduler.get_last_lr()[0]:.1e}',
                'acc': f'{100 * correct_train / total_train:.2f}%'
            })
        
        scheduler.step()
        if torch.isnan(loss):
            break

    print("✅ Finished Training.")

    print("\n[STEP 5/5] Evaluating model on test data...")
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for data in tqdm(testloader, desc="Evaluating", leave=True):
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    accuracy = 100 * correct_test / total_test
    print(f'✅ Evaluation Complete. Accuracy on 10,000 test images: {accuracy:.2f} %')